In [1]:
import rospy
import actionlib
import threading
from IPython.display import display, clear_output
import ipywidgets as widgets
from geometry_msgs.msg import Point
from nav_msgs.msg import Odometry
from assignment2_part1.msg import posit, PlanningAction, PlanningGoal
from sensor_msgs.msg import LaserScan
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time
%matplotlib widget

In [2]:
client = None
odom_sub = None
pos_vel_pub = None
current_position = {'x': 0.0, 'y': 0.0}
distance_to_obstacle = 0.0 
regions_ = None
reached_goals = []
failed_goals = []
global client, odom_sub, pos_vel_pub

x_slider = widgets.FloatSlider(value=0, min=-9, max=9, step=0.5, description='X:')
y_slider = widgets.FloatSlider(value=0, min=-9, max=9, step=0.5, description='Y:')
send_button = widgets.Button(description='Send Goal', button_style='success')
cancel_button = widgets.Button(description='Cancel Goal', button_style='danger')
position_label = widgets.Label(value="Robot Position: (0.0, 0.0)")
obstacle_label = widgets.Label(value="Distance to nearest obstacle: N/A")


def clbk_odom(msg):
    global current_position, pos_vel_pub
    current_position['x'] = msg.pose.pose.position.x
    current_position['y'] = msg.pose.pose.position.y

    # Publish custom posit message
    new_info = posit()
    new_info.x = current_position['x']
    new_info.y = current_position['y']
    new_info.vel_x = msg.twist.twist.linear.x
    new_info.vel_z = msg.twist.twist.angular.z
    pos_vel_pub.publish(new_info)

    # Update the UI
    position_label.value = f"Robot Position: ({new_info.x:.2f}, {new_info.y:.2f})"

def feedback_cb(feedback):
    if feedback.stat == "Target reached!":
        print("Target reached!")
    elif feedback.stat == "Target cancelled!":
        print("Goal cancelled!")

def send_goal_callback(b):
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x_slider.value
    goal.target_pose.pose.position.y = y_slider.value
    print(f"Sending goal to ({goal.target_pose.pose.position.x}, {goal.target_pose.pose.position.y})")
    client.send_goal(goal, feedback_cb=feedback_cb)

def cancel_goal_callback(b):
    print("Cancelling goal...")
    client.cancel_goal()
    
def clbk_laser(msg):
    global regions_
    regions_ = {
        'right':  min(min(msg.ranges[0:143]), 10),
        'fright': min(min(msg.ranges[144:287]), 10),
        'front':  min(min(msg.ranges[288:431]), 10),
        'fleft':  min(min(msg.ranges[432:575]), 10),
        'left':   min(min(msg.ranges[576:719]), 10),
    }
    min_dist, min_dist_dir = get_min_laser(regions_)
    obstacle_label.value = f"Distance to nearest obstacle is {min_dist:.2f} at {min_dist_dir}"

def argmin(lst):
    if not lst:
        return None, None
    min_index = 0
    min_value = lst[0]
    for i in range(1, len(lst)):
        if lst[i] < min_value:
            min_value = lst[i]
            min_index = i
    return min_index, min_value

def get_min_laser(laserdict):
    min_index, min_value = argmin(list(laserdict.values()))
    return min_value, list(laserdict.keys())[min_index]

def feedback_cb(feedback):
    if feedback.stat == "Target reached!":
        print("Target reached!")
        reached_goals.append((current_position['x'], current_position['y']))
    elif feedback.stat == "Target cancelled!":
        print("Goal cancelled!")
        failed_goals.append((current_position['x'], current_position['y']))

send_button.on_click(send_goal_callback)
cancel_button.on_click(cancel_goal_callback)


rospy.init_node('action_client', anonymous=True)
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
print("⏳ Waiting for action server...")
client.wait_for_server()
print("✅ Connected to action server!")

odom_sub = rospy.Subscriber('/odom', Odometry, clbk_odom)
pos_vel_pub = rospy.Publisher('/robot_pos_vel', posit, queue_size=10)
sub_laser = rospy.Subscriber('/scan', LaserScan, clbk_laser)

⏳ Waiting for action server...
✅ Connected to action server!


In [3]:
x_data = []
y_data = []

fig, (ax_trail, ax_summary) = plt.subplots(1, 2, figsize=(10, 5))
sc = ax_trail.scatter([], [], c='red', s=100, label='Current Position')
trail_line, = ax_trail.plot([], [], 'b--', linewidth=1, label='Trail')
reached_scatter = ax_trail.scatter([], [], c='green', s=80, label='Reached Goal')
failed_scatter = ax_trail.scatter([], [], c='red', marker='x', s=80, label='Failed Goal')
ax_trail.set_xlim(-10, 10)
ax_trail.set_ylim(-10, 10)
ax_trail.set_title("Robot Position with Trail")
ax_trail.set_xlabel("X")
ax_trail.set_ylabel("Y")
ax_trail.legend(loc='upper right')

def update_goal_plot():
    ax_summary.clear()
    reached = len(reached_goals)
    failed = len(failed_goals)
    ax_summary.pie(
        [reached, failed],
        labels=["Reached", "Failed"],
        colors=["green", "red"],
        autopct='%1.0f%%',
        startangle=140
    )
    ax_summary.set_title("Goal Outcome Summary")

def update(frame):
    x = current_position['x']
    y = current_position['y']
    x_data.append(x)
    y_data.append(y)

    if len(x_data) > 400:
        x_data[:] = x_data[-400:]
        y_data[:] = y_data[-400:]

    trail_line.set_data(x_data, y_data)
    sc.set_offsets([[x, y]])

    if reached_goals:
        xs, ys = zip(*reached_goals)
        reached_scatter.set_offsets(list(zip(xs, ys)))
    if failed_goals:
        xs, ys = zip(*failed_goals)
        failed_scatter.set_offsets(list(zip(xs, ys)))

    update_goal_plot()

    return trail_line, sc, reached_scatter, failed_scatter

ani = FuncAnimation(fig, update, interval=200)

plot_output = widgets.Output()
with plot_output:
    display(fig)

full_ui = widgets.VBox([
    x_slider,
    y_slider,
    widgets.HBox([send_button, cancel_button]),
    position_label,
    obstacle_label
])

display(full_ui)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Sending goal to (-2.5, -2.5)
Sending goal to (2.0, -2.5)
Sending goal to (2.0, 5.5)
Cancelling goal...
Sending goal to (2.0, 0.0)
Cancelling goal...
